In [15]:
import os
import pandas as pd
import psycopg2
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout

class PGConnector():
    def __init__(self):
        pass
    DB_HOST = os.getenv('DB_HOST')
    DB_PORT = os.getenv('DB_PORT')
    DB_USER = os.getenv('DB_USER')
    DB_DATABASE = os.getenv('DB_DATABASE')
    DB_PASSWORD = os.getenv('DB_PASSWORD')

    def connection(self):
        return psycopg2.connect(
            database=self.DB_DATABASE,
            host=self.DB_HOST,
            user=self.DB_USER,
            password=self.DB_PASSWORD,
            port=self.DB_PORT
        )

postgres = PGConnector()
conn = postgres.connection()


In [3]:
df = pd.read_sql('SELECT * FROM alignable.email_stats', conn)

/tmp/ipykernel_5582/2869797104.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM alignable.email_stats', conn)


In [47]:
df = df.fillna('no_action')
nodes1 = df['action'].drop_duplicates().to_list()
nodes2 = df['event_name'].drop_duplicates().to_list()
nodes3 = df['session_event_name'].drop_duplicates().to_list()


all_nodes = nodes1 + nodes2 + nodes3

all_nodes_unique = list(set(all_nodes))

# get first node values
df[['action', 'event_name', 'sends']].groupby(['action', 'event_name']).sum().reset_index()

,action,event_name,sends
0,connection_request,clicked,22321
1,connection_request,no_action,30575
2,connection_request,opened,35793
3,received_recommendation,clicked,3490
4,received_recommendation,no_action,2455
5,received_recommendation,opened,3822


In [66]:
df

,mailing_id,mailer,action,event_name,session_event_name,sends
0,297147,Notification,connection_request,clicked,connection_request_accepted,15020
1,297147,Notification,connection_request,clicked,recommendation_request_accepted,99
2,297147,Notification,connection_request,clicked,connection_request_sent,1830
3,297147,Notification,connection_request,clicked,conversation_message,1405
4,297147,Notification,connection_request,clicked,recommendation_request_sent,114
5,297147,Notification,connection_request,clicked,no_action,3853
6,297147,Notification,connection_request,opened,no_action,35793
7,297147,Notification,connection_request,no_action,no_action,30575
8,297161,Notification,received_recommendation,clicked,connection_request_accepted,55
9,297161,Notification,received_recommendation,clicked,recommendation_request_accepted,2619


In [95]:
# Set up nodes and links to customize Sankey
nodes = [
    {"id": "CR", "title": "Connection \nRequest"},
    {"id": "RR", "title": "Received \nRec."},
    {"id": "C", "title": "Clicked"},
    {"id": "NA", "title": "No Action"},
    {"id": "O", "title": "Opened"},
    {"id": "CRA", "title": "Connection \n Request \n Accepted"},
    {"id": "CRS", "title": "Connection \nRequest Sent"},
    {"id": "CM", "title": "Conversation \n Message"},
    {"id": "RRS", "title": "Rec. Request \nSent"},
    {"id": "RRA", "title": "Rec. Request \nAccepted"},
]

test = '''

action	event_name	sends
0	connection_request	clicked	22321
1	connection_request	no_action	30575
2	connection_request	opened	35793
3	received_recommendation	clicked	3490
4	received_recommendation	no_action	2455
5	received_recommendation	opened	3822
'''

links_297147 = [
    {"source": "C", "target": "CRA", "value": 15020},
    {"source": "C", "target": "RRA", "value": 99},
    {"source": "C", "target": "CRS", "value": 1830},
    {"source": "C", "target": "CM", "value": 1405},
    {"source": "C", "target": "RRS", "value": 114},
    {"source": "C", "target": "NA", "value": 3853, "color": "gray"},
    {"source": "CR", "target": "NA", "value": 30575, "color": "gray"},
    {"source": "CR", "target": "O", "value": 35793},
    {"source": "O", "target": "C", "value": 22321},
    {"source": "O", "target": "NA", "value": 35793-22321, "color": "gray"},
]

links_297161 = [
    {"source": "O", "target": "C", "value": 3490},
    {"source": "O", "target": "NA", "value": 3822-3490, "color": "gray"},
    {"source": "RR", "target": "O", "value": 3822},
    {"source": "RR", "target": "NA", "value": 2455, "color": "gray"},
    {"source": "C", "target": "CRA", "value": 55},
    {"source": "C", "target": "RRA", "value": 2619},
    {"source": "C", "target": "CRS", "value": 82},
    {"source": "C", "target": "CM", "value": 140},
    {"source": "C", "target": "RRS", "value": 199},
    {"source": "C", "target": "NA", "value": 395, "color": "gray"},
]

In [96]:
layout = Layout(width="1000", height="1000")
SankeyWidget(links=links_297147, nodes=nodes, layout=layout)

SankeyWidget(layout=Layout(height='1000', width='1000'), links=[{'source': 'C', 'target': 'CRA', 'value': 1502…

In [97]:
SankeyWidget(links=links_297161, nodes=nodes, layout=layout)

SankeyWidget(layout=Layout(height='1000', width='1000'), links=[{'source': 'O', 'target': 'C', 'value': 3490},…